<a href="https://colab.research.google.com/github/edquestofficial/Gen-AI-Cohort/blob/main/2024/april/Level_2/RAG_Generative_Search_with_OpenAI_and_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Required Libraries

In [ ]:
!pip install pdfplumber tiktoken openai chromaDB sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Import Required Libraries

In [ ]:
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Extract text from pdf file

In [ ]:
# Function to extract text from a PDF file.
# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list

def load_text_from_pdf(pdf_path):
    full_text = []
    p = 1

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"page {p}"
            text = page.extract_text()

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))

            page_text = " ".join(lines)
            full_text.append([page_no, page_text])
            p += 1

    return full_text

In [ ]:
pdf_directory_str = "/content/drive/MyDrive/Gen AI Course/RAG_For_HDFC_Policy/data"
pdf_directory_path = Path(pdf_directory_str)

In [ ]:
pdf_directory_path

PosixPath('/content/drive/MyDrive/Gen AI Course/RAG_For_HDFC_Policy/data')

In [ ]:
# Function to check whether a word is present in a table or not for segregation of regular text and tables

def check_bboxes(word, table_bbox):
    # Check whether word is inside a table bbox.
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

In [ ]:
# Define the directory containing the PDF files
pdf_directory = Path(pdf_directory_path)

# Initialize an empty list to store the extracted texts and document names
data = []

# Loop through all files in the directory
for pdf_path in pdf_directory.glob("*.pdf"):

    # Process the PDF file
    print(f"...Processing {pdf_path.name}")

    # Call the function to extract the text from the PDF
    extracted_text = load_text_from_pdf(pdf_path)

    # Convert the extracted list to a PDF, and add a column to store document names
    extracted_text_df = pd.DataFrame(extracted_text, columns=['Page No.', 'Page_Text'])
    extracted_text_df['Document Name'] = pdf_path.name

    # Append the extracted text and document name to the list
    data.append(extracted_text_df)

    # Print a message to indicate progress
    print(f"Finished processing {pdf_path.name}")

# Print a message to indicate all PDFs have been processed
print("All PDFs have been processed.")

...Processing HDFC-Life-Easy-Health-101N110V03-Policy-Bond-Single-Pay.pdf
Finished processing HDFC-Life-Easy-Health-101N110V03-Policy-Bond-Single-Pay.pdf
All PDFs have been processed.


In [ ]:
data

[   Page No.                                          Page_Text  \
 0    page 1  Part A <<Date>> <<Policyholder’s Name>> <<Poli...   
 1    page 2  Agency/Intermediary Contact Details: <<Agency/...   
 2    page 3  POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ...   
 3    page 4  [[null, "<< dd/mm/yyyy >>"], ["Appointee's Add...   
 4    page 5  Part B Definitions The following capitalised t...   
 5    page 6  [["", "", "iii. A rise in cardiac biomarkers o...   
 6    page 7  [["", "", "apart; and\nii. Requiring continuou...   
 7    page 8  [["", "Sclerosis\nwith\npersisting\nsymptoms",...   
 8    page 9  (16) Intensive Care Unit (ICU) - means an iden...   
 9   page 10  [["Plan option", "Benefits covered"], ["A", "D...   
 10  page 11  v. In case 100% of the Sum Insured has been us...   
 11  page 12                                                      
 12  page 13  Part D 1. Claims Procedure You have the option...   
 13  page 14  In case you are not agreeable to any of the pr..

In [ ]:
insurance_pdfs_data = pd.concat(data, ignore_index=True)

In [ ]:
insurance_pdfs_data.head()

,Page No.,Page_Text,Document Name
0,page 1,Part A <<Date>> <<Policyholder’s Name>> <<Poli...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...
1,page 2,Agency/Intermediary Contact Details: <<Agency/...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...
2,page 3,POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...
3,page 4,"[[null, ""<< dd/mm/yyyy >>""], [""Appointee's Add...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...
4,page 5,Part B Definitions The following capitalised t...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...


## Let's also check the length of all the texts as there might be some empty pages or pages with very few words that we can drop

In [ ]:
insurance_pdfs_data['Text_Length'] = insurance_pdfs_data['Page_Text'].apply(lambda x: len(x.split(' ')))

In [ ]:
insurance_pdfs_data.head()

,Page No.,Page_Text,Document Name,Text_Length
0,page 1,Part A <<Date>> <<Policyholder’s Name>> <<Poli...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,508
1,page 2,Agency/Intermediary Contact Details: <<Agency/...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,85
2,page 3,POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,298
3,page 4,"[[null, ""<< dd/mm/yyyy >>""], [""Appointee's Add...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,63
4,page 5,Part B Definitions The following capitalised t...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,514


## Retain only the rows with a text length of at least 10

In [ ]:
insurance_pdfs_data = insurance_pdfs_data.loc[insurance_pdfs_data['Text_Length'] >= 10]
insurance_pdfs_data

,Page No.,Page_Text,Document Name,Text_Length
0,page 1,Part A <<Date>> <<Policyholder’s Name>> <<Poli...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,508
1,page 2,Agency/Intermediary Contact Details: <<Agency/...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,85
2,page 3,POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,298
3,page 4,"[[null, ""<< dd/mm/yyyy >>""], [""Appointee's Add...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,63
4,page 5,Part B Definitions The following capitalised t...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,514
5,page 6,"[["""", """", ""iii. A rise in cardiac biomarkers o...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,526
6,page 7,"[["""", """", ""apart; and\nii. Requiring continuou...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,569
7,page 8,"[["""", ""Sclerosis\nwith\npersisting\nsymptoms"",...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,195
8,page 9,(16) Intensive Care Unit (ICU) - means an iden...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,783
9,page 10,"[[""Plan option"", ""Benefits covered""], [""A"", ""D...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,601


## Generate and Store Embeddings using OpenAI and ChromaDB

In [ ]:
# Set the API key
api_key_path = "/content/drive/MyDrive/Gen AI Course/RAG_For_HDFC_Policy/api_key.txt"

with open(api_key_path, "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [ ]:
# Import the OpenAI Embedding Function into chroma

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [ ]:
# Define the path where chroma collections will be stored

chroma_data_path = '/content/drive/MyDrive/Gen AI Course/RAG_For_HDFC_Policy/ChromaDB_Data'

In [ ]:
import chromadb

chroma_client = chromadb.PersistentClient()

In [ ]:
# Set up the embedding function using the OpenAI embedding model

model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [ ]:
# Initialise a collection in chroma and pass the embedding_function to it so that it used OpenAI embeddings to embed the documents

insurance_collection = chroma_client.get_or_create_collection(name='RAG_on_Insurance', embedding_function=embedding_function)

In [ ]:
insurance_pdfs_data.head()

,Page No.,Page_Text,Document Name,Text_Length
0,page 1,Part A <<Date>> <<Policyholder’s Name>> <<Poli...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,508
1,page 2,Agency/Intermediary Contact Details: <<Agency/...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,85
2,page 3,POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,298
3,page 4,"[[null, ""<< dd/mm/yyyy >>""], [""Appointee's Add...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,63
4,page 5,Part B Definitions The following capitalised t...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,514


In [ ]:
# Store the metadata for each page in a separate column

insurance_pdfs_data['Metadata'] = insurance_pdfs_data.apply(lambda x: {'Policy_Name': x['Document Name'][:-4], 'Page_No.': x['Page No.']}, axis=1)

In [ ]:
insurance_pdfs_data.head()

,Page No.,Page_Text,Document Name,Text_Length,Metadata
0,page 1,Part A <<Date>> <<Policyholder’s Name>> <<Poli...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,508,{'Policy_Name': 'HDFC-Life-Easy-Health-101N110...
1,page 2,Agency/Intermediary Contact Details: <<Agency/...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,85,{'Policy_Name': 'HDFC-Life-Easy-Health-101N110...
2,page 3,POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,298,{'Policy_Name': 'HDFC-Life-Easy-Health-101N110...
3,page 4,"[[null, ""<< dd/mm/yyyy >>""], [""Appointee's Add...",HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,63,{'Policy_Name': 'HDFC-Life-Easy-Health-101N110...
4,page 5,Part B Definitions The following capitalised t...,HDFC-Life-Easy-Health-101N110V03-Policy-Bond-S...,514,{'Policy_Name': 'HDFC-Life-Easy-Health-101N110...


In [ ]:
# Convert the page text and metadata from your dataframe to lists to be able to pass it to chroma

documents_list = insurance_pdfs_data["Page_Text"].tolist()
metadata_list = insurance_pdfs_data["Metadata"].tolist()


In [ ]:
# Add the documents and metadata to the collection alongwith generic integer IDs. You can also feed the metadata information as IDs by combining the policy name and page no.

insurance_collection.add(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)

In [ ]:
# Let's take a look at the first few entries in the collection

insurance_collection.get(
    ids = ['0'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0'],
 'embeddings': [[0.006147026550024748,
   0.015591486357152462,
   -0.0021446587052196264,
   -0.04094921424984932,
   0.0033571457024663687,
   0.04572618007659912,
   0.0028363235760480165,
   0.0025443979538977146,
   -0.021363653242588043,
   -0.001598127419129014,
   0.031023740768432617,
   0.015485331416130066,
   -0.004120133351534605,
   0.014941288158297539,
   -0.003549551125615835,
   0.017847275361418724,
   0.0240042544901371,
   -0.020872687920928,
   0.025477152317762375,
   0.00047728195204399526,
   -0.0023768723476678133,
   0.021204421296715736,
   -0.002177832182496786,
   0.0262998528778553,
   -0.009759606793522835,
   0.0014024043921381235,
   0.02046133764088154,
   -0.005848465953022242,
   -0.0012970790266990662,
   -0.0011900948593392968,
   -0.003305726917460561,
   0.0028794489335268736,
   -0.027494093403220177,
   -0.007145544979721308,
   -0.008897099643945694,
   0.0059778424911201,
   -0.014078781008720398,
   -0.020793071016669273,
   

## Semantic Search

In [ ]:
# Read the user query

user_query = "Tell me about hdfc life easy health policy"

In [ ]:
def get_semantic_search(collection, query):
  results = collection.query(
    query_texts=user_query,
    n_results=6
  )
  ids = results["ids"][0]
  documents = results["documents"][0]
  distances = results["distances"][0]
  metadatas = results["metadatas"][0]

  results_df = pd.DataFrame({
    'IDs': ids,
    'Documents': documents,
    'Distances': distances,
    'Metadatas': metadatas
  })

  top_3_semantic = results_df.sort_values(by='Distances')
  top_3_semantic = top_3_semantic[:3]
  return top_3_semantic


In [ ]:
top_3_semantic = get_semantic_search(insurance_collection, user_query)

In [ ]:
top_3_semantic

,IDs,Documents,Distances,Metadatas
0,2,POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ...,0.244589,"{'Page_No.': 'page 3', 'Policy_Name': 'HDFC-Li..."
1,0,Part A <<Date>> <<Policyholder’s Name>> <<Poli...,0.273111,"{'Page_No.': 'page 1', 'Policy_Name': 'HDFC-Li..."
2,1,Agency/Intermediary Contact Details: <<Agency/...,0.355762,"{'Page_No.': 'page 2', 'Policy_Name': 'HDFC-Li..."


In [ ]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
                {"role": "user", "content": f"""You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of insurance documents in the dataframe '{results_df}'. These search results are essentially one page of an insurance document that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual text from the policy document and the column 'metadata' contains the policy name and source page. The text inside the document may also contain tables in the format of a list of lists where each of the nested lists indicates a row.

                                                Use the documents in '{results_df}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant policy names and page numbers as citations.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular in format.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the policy name(s) and page numbers(s) as citation.
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

In [ ]:
# Generate the response

response = generate_response(user_query, top_3_semantic)

In [ ]:
response

['The HDFC Life Easy Health Policy is a comprehensive insurance policy offered by HDFC Life. This policy provides coverage for various health-related expenses such as hospitalization, medical treatments, and other health-related costs. Specific details of the policy coverage, benefits, exclusions, and terms can be found in the policy document provided by HDFC Life.',
 '',
 'Citation:',
 '- Policy Name: HDFC Life Easy Health Policy',
 '- Source Page: Page 3',
 '',
 'For detailed information regarding the specific coverage, benefits, and exclusions of the HDFC Life Easy Health Policy, please refer to page 3 of the policy document titled "POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ..." in the provided corpus of insurance documents.']

In [ ]:
print("\n".join(response))

The HDFC Life Easy Health Policy is a comprehensive insurance policy offered by HDFC Life. This policy provides coverage for various health-related expenses such as hospitalization, medical treatments, and other health-related costs. Specific details of the policy coverage, benefits, exclusions, and terms can be found in the policy document provided by HDFC Life.

Citation:
- Policy Name: HDFC Life Easy Health Policy
- Source Page: Page 3

For detailed information regarding the specific coverage, benefits, and exclusions of the HDFC Life Easy Health Policy, please refer to page 3 of the policy document titled "POLICY DOCUMENT- HDFC LIFE EASY HEALTH Unique ..." in the provided corpus of insurance documents.


In [ ]:
# Generate the response
user_query = "Tell me how this policy is useful for cancer patients"

In [ ]:
top_3_semantic = get_semantic_search(insurance_collection, user_query)

In [ ]:
response = generate_response(user_query, top_3_semantic)

In [ ]:
print("\n".join(response))

This policy provides coverage for routine eye tests, dental treatment, and other specified benefits. Unfortunately, there is no specific information related to cancer patients in the search results available. I recommend referring to the policy documents, particularly sections related to health coverage and benefits, to find detailed information about coverage for cancer patients.

Citations for further reference:
- Policy Name: HDFC-Life Insurance Policy
- Relevant Pages: Page 11, Page 17


## Ask new query

In [ ]:
user_query = "what are Surgical Benefit"

In [ ]:
top_3_semantic = get_semantic_search(insurance_collection, user_query)

In [ ]:
response = generate_response(user_query, top_3_semantic)

In [ ]:
print("\n".join(response))

The Surgical Benefit in the insurance policy refers to the coverage provided for surgical procedures related to medical treatments or interventions. This benefit typically includes coverage for the surgical expenses incurred during hospitalization for various medical conditions. Specific details regarding the extent of coverage, eligible procedures, and any limitations would be outlined in the policy document.

Citations:
- Policy Name: HDFC-Life Insurance Policy
- Relevant Page Numbers: Page 11, Page 10, Page 17
